In [3]:
import numpy as np
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import random


def genetic_algorithm(total_set, nvar, objective_function, ngen=50, mu=20,
                      lam=40, cxpb=0.7, mutpb=0.2, tournsize=3,
                      verbose=False):
    # Creates a new class name set_mod which is based on the standard python
    # set. This means set_mod is just like set, with the addion of a fitness
    # attribute.
    creator.create("Fitness", base.Fitness, weights=(-1.0,))
    creator.create("Individual", set, fitness=creator.Fitness)
    set_mod = creator.Individual

    def random_samp(size):
        """Function to initlize individual in the population."""
        return set_mod(random.sample(total_set, size))

    def evaluation(individual):
        """Evaluate the objective function."""
        return objective_function(list(individual)),

    def cxSet(ind1, ind2):
        """Apply a crossover operation on two sets."""
        full_set = list(ind1 | ind2)
        ind1 = set_mod(random.sample(full_set, nvar))
        ind2 = set_mod(random.sample(full_set, nvar))
        return ind1, ind2

    def mutSet(individual):
        """Mutation that randomly removes and item and randomly adds an item.
        """
        temp_set = set_mod(random.sample(individual, nvar-1))
        set_to_choose = np.array(list(temp_set ^ total_set))
        new = random.choice(set_to_choose)
        temp_set.add(new)
        return temp_set,

    toolbox = base.Toolbox()

    # set up the population
    toolbox.register("individual", random_samp, nvar)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # set up the GA functions
    toolbox.register("evaluate", evaluation)
    toolbox.register("mate", cxSet)
    toolbox.register("mutate", mutSet)
    # toolbox.register("select", tools.selNSGA2)
    toolbox.register("select", tools.selTournament, tournsize=tournsize)

    # initialize the population
    pop = toolbox.population(n=mu)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)
    # run the GA
    algorithms.eaMuPlusLambda(pop, toolbox, mu, lam, cxpb, mutpb, ngen, stats,
                              halloffame=hof, verbose=verbose)
    return pop, hof, stats

In [8]:
# import our libraries
import numpy as np
import matplotlib.pyplot as plt
import pwlf
from time import time

# generate sin wave data
n = 100
x = np.linspace(0, 10, num=n)
y = np.sin(x * np.pi / 2)
# add noise to the data
y = np.random.normal(0, 0.05, n) + y

# initialize piecewise linear fit with your x and y data
my_pwlf = pwlf.PiecewiseLinFit(x, y)

number_of_line_segments = 2
t0 = time()
res0 = my_pwlf.fitfast(number_of_line_segments)
t1 = time()
print('run time fitfast:', t1-t0, '(s)')
print('ssr:', my_pwlf.ssr)

t2 = time()
my_pwlf.use_custom_opt(number_of_line_segments)
total_set = set(my_pwlf.x_data)
total_set.remove(x.min())
total_set.remove(x.max())
pop, hof, stats = genetic_algorithm(total_set, my_pwlf.nVar,
                                         my_pwlf.fit_with_breaks_opt, ngen=20,
                                         mu=125, lam=250, cxpb=0.7, mutpb=0.2,
                                         tournsize=5, verbose=False)
ssr = my_pwlf.fit_with_breaks_opt(list(hof[0]))
t3 = time()
print('run time ga:', t3-t2, '(s)')
print('ssr:', ssr)

print(my_pwlf.fit_breaks)

run time fitfast: 0.03260636329650879 (s)
ssr: 38.988008479001294
run time ga: 1.536201000213623 (s)
ssr: 38.98802716435883
[ 0.          2.72727273 10.        ]
